# Playground

## Import Libraries

In [72]:
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as offline
import os
import numpy as np
from plotly.offline import plot
from pathlib import Path

In [73]:
offline.init_notebook_mode(connected=True)

# Heatmap of GHG Emissions Scopes by Sector

In [74]:
scope3_completeness = False

In [75]:
path = Path(os.path.dirname (os.getcwd()))
XLSX_PATH = os.path.join(path.parent.parent, 'sp100.xlsx')

COLS_TO_USE = {
'ghg_quant':['company_id', 'ghg_scope_1','ghg_loc_scope_2','ghg_mkt_scope_2', 'ghg_scope3_total', 'reporting_year', "Source"],
'grouping':['company_id', 'Sector1'],
'companies':['company_id', 'SP100'],
'ghg_qual':['company_id', 'scope3_completeness']
}

all_df = []
for sheetname, cols in COLS_TO_USE.items():
    all_df.append(pd.read_excel(
        XLSX_PATH, 
        sheet_name = sheetname,
        engine = 'openpyxl', 
        usecols = cols
        ))
merged_df = all_df[0]
for i in range(len(COLS_TO_USE) - 1):
    merged_df = pd.merge(
        left = merged_df,
        right = all_df[i+1],
        how="left",
        on="company_id"
    )
# Select Final Figures and Year 2019
cond1 = (merged_df['reporting_year'] == 2019)
cond2 = (merged_df['Source']== 'Final')
cond3 = (merged_df['SP100'] == 1)
filter_cond = cond1 & cond2 & cond3
if scope3_completeness : 
    cond4 = (merged_df['scope3_completeness'] == "full")
    filter_cond = filter_cond & cond4

merged_df = merged_df.loc[filter_cond]

#a =  merged_df[merged_df['company_id']==company_id]['Sector1']

In [76]:
merged_df


,company_id,Source,reporting_year,ghg_scope_1,ghg_loc_scope_2,ghg_mkt_scope_2,ghg_scope3_total,Sector1,SP100,scope3_completeness
6,1.0,Final,2019.0,4050000,1780000,1320000,8925800.0,Industrials,1.0,full
18,2.0,Final,2019.0,533000,518000,439000,14073000.0,Healthcare,1.0,full
30,3.0,Final,2019.0,314421,308204,249777,1334209.0,Healthcare,1.0,no
42,4.0,Final,2019.0,18923,281489,214680,932653.0,Technology,1.0,full
54,5.0,Final,2019.0,11816,56113,43526,544093.0,Technology,1.0,full
...,...,...,...,...,...,...,...,...,...,...
1194,100.0,Final,2019.0,91993,771327,4988,3610451.0,Financial Services,1.0,full
1206,101.0,Final,2019.0,0,0,0,0.0,Consumer Discretionary,1.0,no
1242,104.0,Final,2019.0,0,0,0,0.0,Technology,1.0,no
1338,112.0,Final,2019.0,37175,1393730,729490,3725393.0,Communication Services,1.0,full


In [77]:
scope1 = pd.to_numeric(merged_df['ghg_scope_1'], errors = 'coerce')

merged_df['ghg_loc_scope_2'] = pd.to_numeric(merged_df['ghg_loc_scope_2'], errors = 'coerce')
merged_df['ghg_mkt_scope_2'] = pd.to_numeric(merged_df['ghg_mkt_scope_2'], errors = 'coerce')
scope2 = merged_df[['ghg_loc_scope_2','ghg_mkt_scope_2']].min(axis=1)
#merged_df = merged_df.assign(ghg_scope_2 = scope2)
scope3 = merged_df['ghg_scope3_total']

total_scope = scope1 + scope2 + scope3

scope1_dist = scope1 / total_scope
scope2_dist = scope2 / total_scope
scope3_dist = scope3 / total_scope

#total_scope = merged_df.assign(ghg_total_scope = merged_df['ghg_scope_1'] + scope2 + merged_df['ghg_scope3_total'])
merged_df = merged_df.assign(scope1_dist = scope1_dist)
merged_df = merged_df.assign(scope2_dist = scope2_dist)
merged_df = merged_df.assign(scope3_dist = scope3_dist)



In [78]:
fields = ['scope1_dist','scope2_dist', 'scope3_dist', 'Sector1']
scope_by_sector_df = merged_df [fields]

scope_by_sector_df = scope_by_sector_df.dropna().sort_values("Sector1", ascending=True)
#data.sort_values("Salary", axis = 0, ascending = True,
#                 inplace = True, na_position ='first')

In [79]:
scope_by_sector_df

,scope1_dist,scope2_dist,scope3_dist,Sector1
822,0.989145,0.010855,0.000000,Communication Services
1338,0.008276,0.162395,0.829329,Communication Services
78,0.016585,0.197532,0.785883,Communication Services
510,0.011696,0.000000,0.988304,Communication Services
1074,0.478599,0.521401,0.000000,Communication Services
...,...,...,...,...
1050,0.471761,0.514795,0.013443,Technology
438,0.829170,0.000050,0.170780,Utilities
834,1.000000,0.000000,0.000000,Utilities
1014,0.713702,0.000298,0.286000,Utilities


In [80]:
total = scope_by_sector_df.mean(numeric_only=True).rename('Total')
total['Sector1'] = "Total"

In [81]:
total

scope1_dist    0.188779
scope2_dist    0.149928
scope3_dist    0.661293
Sector1           Total
Name: Total, dtype: object

In [82]:
scope_by_sector_df = scope_by_sector_df.append(total, ignore_index=True)

In [83]:
scope_by_sector_df

,scope1_dist,scope2_dist,scope3_dist,Sector1
0,0.989145,0.010855,0.000000,Communication Services
1,0.008276,0.162395,0.829329,Communication Services
2,0.016585,0.197532,0.785883,Communication Services
3,0.011696,0.000000,0.988304,Communication Services
4,0.478599,0.521401,0.000000,Communication Services
...,...,...,...,...
90,0.829170,0.000050,0.170780,Utilities
91,1.000000,0.000000,0.000000,Utilities
92,0.713702,0.000298,0.286000,Utilities
93,0.094668,0.049513,0.855818,Utilities


In [84]:
scope_by_sector_df = scope_by_sector_df.groupby('Sector1', sort=False).mean()

In [85]:
scope_by_sector_df

,scope1_dist,scope2_dist,scope3_dist
Sector1,,,
Communication Services,0.318732,0.250043,0.431225
Consumer Discretionary,0.058529,0.011029,0.930442
Consumer Staples,0.068022,0.076355,0.855623
Energy,0.557519,0.001110,0.441371
Financial Services,0.038153,0.179519,0.782329
Healthcare,0.140062,0.104649,0.755290
Industrials,0.352629,0.232005,0.415367
Materials,0.365151,0.283511,0.351338
Real Estate,0.129314,0.524160,0.346527


In [86]:
scope_by_sector_df


,scope1_dist,scope2_dist,scope3_dist
Sector1,,,
Communication Services,0.318732,0.250043,0.431225
Consumer Discretionary,0.058529,0.011029,0.930442
Consumer Staples,0.068022,0.076355,0.855623
Energy,0.557519,0.001110,0.441371
Financial Services,0.038153,0.179519,0.782329
Healthcare,0.140062,0.104649,0.755290
Industrials,0.352629,0.232005,0.415367
Materials,0.365151,0.283511,0.351338
Real Estate,0.129314,0.524160,0.346527


In [87]:
#CHECK
scope_by_sector_df['scope1_dist'] + scope_by_sector_df['scope2_dist'] + scope_by_sector_df['scope3_dist']

Sector1
Communication Services    1.0
Consumer Discretionary    1.0
Consumer Staples          1.0
Energy                    1.0
Financial Services        1.0
Healthcare                1.0
Industrials               1.0
Materials                 1.0
Real Estate               1.0
Technology                1.0
Utilities                 1.0
Total                     1.0
dtype: float64

In [88]:
x0 = scope_by_sector_df.index
y0=['Scope 1', 'Scope 2', 'Scope 3']
z0 = [scope_by_sector_df['scope1_dist'],
      scope_by_sector_df['scope2_dist'],
      scope_by_sector_df['scope3_dist']]

In [89]:
trace = go.Heatmap(
    z = z0,
    x = x0,
    y = y0,
    colorbar = dict(tickformat = "%"),
    colorscale = 'amp')

In [90]:
list(scope_by_sector_df.index)

['Communication Services',
 'Consumer Discretionary',
 'Consumer Staples',
 'Energy',
 'Financial Services',
 'Healthcare',
 'Industrials',
 'Materials',
 'Real Estate',
 'Technology',
 'Utilities',
 'Total']

In [91]:
if scope3_completeness:
    title = "Real Picture Based on Transparent Companies"
else:
    title = "What is Currently Reported"
title = ""

In [92]:
layout = go.Layout (
    title = title,
    titlefont = dict(family = 'Arial', size = 25),
    plot_bgcolor = 'antiquewhite',
)

In [93]:
fig = go.Figure(data = [trace], 
                layout = layout)
offline.iplot(fig)

In [94]:
config = {'displaylogo': False,
        'responsive': True
        }
if scope3_completeness:
    name_fig = 'real_scope_heatmap'
else:
    name_fig = 'reported_scope_heatmap'

fig.write_image(
                "../../django_project/static/django_project/images/charts/{}/{}.jpeg".format(
                    'general',
                    name_fig),
                scale=1,
                )
plot(
    fig,
    config=config,
    filename = '../../django_project/templates/django_project/home/charts/html_exports/{}/{}.html'.format(
        'general',
        name_fig),
    auto_open=False)

'../../django_project/templates/django_project/home/charts/html_exports/general/reported_scope_heatmap.html'

In [95]:
print (scope3_completeness)

False
